This notebook contains an small code using the [Python package](https://rileyjmurray.github.io/sageopt/) `sageopt` to check the copositivity of the polynomial from Example 4.7 from the paper "Copositivity, discriminants and nonseparable signed supports" by Elisenda Feliu, Joan Ferrer and Máté L. Telek.

In [3]:
import time
import math
import numpy as np
import sageopt as so
import warnings
warnings.filterwarnings("ignore", module="mosek")  # optional: quiet MOSEK chatter

In [ ]:
def certify_copositivity_via_sage(f, X=None, solver_order=("MOSEK","ECOS"), verbose=False):
    """
    Decide copositivity via sage_feasibility. Success iff prob.value > -np.inf.
    Returns: (ok, solver_used, status, value)
    """
    prob = so.sage_feasibility(f, X)
    solver_used, status, value = None, "failed", float("-inf")
    for s in solver_order:
        try:
            status, val = prob.solve(verbose=verbose, solver=s)  # (status, value)
            value = prob.value if val is None else val
            solver_used = s
            break
        except Exception:
            continue
    ok = (prob.value is not None) and np.isfinite(prob.value) and (prob.value > -np.inf)
    return ok, solver_used, status, float(value)


# Build f on the positive orthant via signomials: y[i](x) = exp(x[i])
y = so.standard_sig_monomials(4)
d = ((10/9)**(9/10) * 40**(1/10))  #circuit number
e = 1e-7 #small perturbation
f = 1 + y[0]**40 + y[1]**40 + y[2]**40 + y[3]**40 - (d+e)*(y[0]*y[1]*y[2]*y[3])

#We do not time the first call
ok, solver_used, status, val = certify_copositivity_via_sage(f) 

# Start measuring time
start_time = time.time()
ok, solver_used, status, val = certify_copositivity_via_sage(f)
end_time = time.time()

print("SAGE:", ok, status, val)
print(f"Time taken: {end_time - start_time} seconds")

SAGE: True solved -0.0
Time taken: 0.011145830154418945 seconds


`sageopt` wrongly detects f as a SAGE polynomial, and in particular, as copositive.